In [1]:
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import col 
from pyspark.sql.types import *
from pyspark.sql.functions import udf

In [2]:
spark = (SparkSession
          .builder\
          .master("local[4]")
          .appName("Q4")\
          #.config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.ContainerCredentialsProvider")\
          .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.EnvironmentVariableCredentialsProvider")\
          .config("spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem")\
          .config("spark.sql.shuffle.partitions", "2")\
          .getOrCreate())

In [242]:
s3url = 's3a://dtt-rookie-battlefiled/practice/Q4/*.csv'
df = spark.read.format("csv") \
    .option("innerSchema","true") \
    .option("header","true") \
    .option("nullValue","None") \
    .load(s3url)


In [17]:
df.printSchema()

root
 |-- 鄉鎮市區: string (nullable = true)
 |-- 交易標的: string (nullable = true)
 |-- 土地區段位置建物區段門牌: string (nullable = true)
 |-- 土地移轉總面積平方公尺: string (nullable = true)
 |-- 都市土地使用分區: string (nullable = true)
 |-- 非都市土地使用分區: string (nullable = true)
 |-- 非都市土地使用編定: string (nullable = true)
 |-- 交易年月日: string (nullable = true)
 |-- 交易筆棟數: string (nullable = true)
 |-- 移轉層次: string (nullable = true)
 |-- 總樓層數: string (nullable = true)
 |-- 建物型態: string (nullable = true)
 |-- 主要用途: string (nullable = true)
 |-- 主要建材: string (nullable = true)
 |-- 建築完成年月: string (nullable = true)
 |-- 建物移轉總面積平方公尺: string (nullable = true)
 |-- 建物現況格局-房: string (nullable = true)
 |-- 建物現況格局-廳: string (nullable = true)
 |-- 建物現況格局-衛: string (nullable = true)
 |-- 建物現況格局-隔間: string (nullable = true)
 |-- 有無管理組織: string (nullable = true)
 |-- 總價元: string (nullable = true)
 |-- 單價元平方公尺: string (nullable = true)
 |-- 車位類別: string (nullable = true)
 |-- 車位移轉總面積平方公尺: string (nullable = true)
 |-- 車位總價元: string (nullabl

In [227]:
# Data Mining
df.select('主要用途').distinct().show()
df.select('鄉鎮市區').where(col('鄉鎮市區').rlike(expr_taipei)).distinct().show()
df.select('建築完成年月').show()

+--------------+
|      主要用途|
+--------------+
|        住商用|
|        商業用|
|        工業用|
|見其它登記事項|
|          農舍|
|        辦公室|
|見其他登記事項|
|        住家用|
|        工商用|
|    見使用執照|
|      國民住宅|
|        住工用|
|          住宅|
+--------------+

+--------+
|鄉鎮市區|
+--------+
|  中正區|
|  中山區|
|  內湖區|
|  士林區|
|  北投區|
|  大安區|
|  文山區|
|  大同區|
|  萬華區|
|  南港區|
|  松山區|
|  信義區|
+--------+

+------------+
|建築完成年月|
+------------+
|     1070702|
|     0691002|
|     0840311|
|     0850522|
|     0611127|
|     0950817|
|     0750228|
|     0970218|
|     1071213|
|     1071213|
|     1071213|
|     1071213|
|     1071213|
|     1071213|
|     1071213|
|     1071213|
|     0620410|
|     1071213|
|     1071213|
|     1071213|
+------------+
only showing top 20 rows



In [223]:
#只住人
expr_home='住宅|國民住宅|住家用'
#Taipei
expr_taipei='南港區|北投區|內湖區|萬華區|中正區|文山區|中山區|松山區|信義區|大安區|大同區|士林區'



In [224]:
#坪＊0.3025 = 平方公尺 
a = 1/0.3025

In [18]:
df.createOrReplaceTempView("DF")

In [288]:
sql = f"""
        SELECT `編號`, `鄉鎮市區`, `主要用途`, 
        ROUND(CAST(`單價元平方公尺` AS INT)*/0.3025),2) AS `單價元坪`, 
        CAST(SUBSTRING(`建築完成年月`,1,3) AS INT)+1911 AS `建築完成年`
        FROM DF
        WHERE
        `編號` != 'serial number' AND
        `主要用途` REGEXP '{expr_home}' AND
        `鄉鎮市區` REGEXP '{expr_taipei}' AND
        CAST(`單價元平方公尺` AS INT)*(1/0.3025) BETWEEN 200000 and 500000 AND
        (CAST(SUBSTRING(`建築完成年月`,1,3) AS INT)+1911) >= 2000
"""

In [289]:
result = spark.sql(sql)

In [290]:
result.show()

+-------------------+--------+--------+---------+----------+
|               編號|鄉鎮市區|主要用途| 單價元坪|建築完成年|
+-------------------+--------+--------+---------+----------+
|RPTNMLOLNHOFFBA18CA|  萬華區|  住家用|484879.32|      2011|
|RPUTMLNJNHOFFCA68CA|  中山區|  住家用|483778.49|      2011|
|RPTNMLNJNHOFFDA08CA|  南港區|  住家用|498251.22|      2011|
|RPUOMLNKNHOFFAA18DA|  文山區|  住家用|446155.36|      2010|
|RPXNMLMJNHOFFBA28DA|  萬華區|  住家用|466181.80|      2011|
|RPVNMLOKNHOFFCA38CA|  中山區|  住家用|433831.39|      2003|
|RPPNMLSLNHOFFBA18CA|  萬華區|  住家用|483999.98|      2011|
|RPOQMLOLNHOFFCA08CA|  內湖區|  住家用|459100.81|      2006|
|RPRPMLRJNHOFFEA58CA|  士林區|  住家用|451302.46|      2002|
|RPRNMLLKNHOFFDA87DA|  南港區|  住家用|420413.21|      2008|
|RPSQMLQJNHOFFCA48CA|  內湖區|  住家用|497312.38|      2009|
|RPUOMLKKNHOFFAA77CA|  文山區|  住家用|433930.56|      2008|
|RPROMLTJNHOFFAA28CA|  文山區|  住家用|481355.35|      2011|
+-------------------+--------+--------+---------+----------+



In [222]:
# UDF f.substring(col('建築完成年月'),0,3)
def toAD (x):
    if x is not None:
        return f.substring(x,0,3) + 1911


In [243]:
df.select(col('編號'),
          col('鄉鎮市區'),
          col('主要用途'),
          (col('單價元平方公尺')*a).alias('單價元坪'),
          col('建築完成年月')) \
    .where(col('編號') != 'serial number') \
    .where(col('鄉鎮市區').rlike(expr_taipei)) \
    .where(col('主要用途').rlike(expr_home)) \
    .where(toAD(col('建築完成年月'))>1999) \
    .where((col('單價元平方公尺')*a)>=200000) \
    .where((col('單價元平方公尺')*a)<=500000) \
    .show()

+-------------------+--------+--------+------------------+------------+
|               編號|鄉鎮市區|主要用途|          單價元坪|建築完成年月|
+-------------------+--------+--------+------------------+------------+
|RPTNMLOLNHOFFBA18CA|  萬華區|  住家用| 484879.3388429752|     1000908|
|RPUTMLNJNHOFFCA68CA|  中山區|  住家用| 483778.5123966942|     1000824|
|RPTNMLNJNHOFFDA08CA|  南港區|  住家用| 498251.2396694215|     1000307|
|RPUOMLNKNHOFFAA18DA|  文山區|  住家用|446155.37190082646|     0990520|
|RPXNMLMJNHOFFBA28DA|  萬華區|  住家用| 466181.8181818182|     1000908|
|RPVNMLOKNHOFFCA38CA|  中山區|  住家用| 433831.4049586777|     0920618|
|RPPNMLSLNHOFFBA18CA|  萬華區|  住家用|          484000.0|     1000908|
|RPOQMLOLNHOFFCA08CA|  內湖區|  住家用|  459100.826446281|     0951027|
|RPRPMLRJNHOFFEA58CA|  士林區|  住家用|451302.47933884297|     0910131|
|RPRNMLLKNHOFFDA87DA|  南港區|  住家用| 420413.2231404959|     0970122|
|RPSQMLQJNHOFFCA48CA|  內湖區|  住家用| 497312.3966942149|     0980106|
|RPUOMLKKNHOFFAA77CA|  文山區|  住家用| 433930.5785123967|     0970627|
|RPROMLTJNHO

In [31]:
spark.stop()